#Generating Polyphonic Piano Samples Using Google Magenta
This Colab file shows generated samples from multiple Mageneta models trained by me or using pre-trained bundle files found on the Magenta github. These models are exhibited with and without conditioning on a set of Ludwig van Beethoven piano samples.

1. Pre-Trained Polyphony Rnn 
2. Pre-Trained Performance Rnn with "performance" Configuration Conditioned on a dataset of Beethoven Midi
3. Performance Rnn with "performance_with_dynamics_compact" Configuration
  - Initially trained on the entire MAESTRO dataset
  - With and without conditioning on Beethoven Dataset
4. Pre-trained Performance Rnn with "performance_with_dynamics" Configuration
  - Model initially trained by Google Brain Magenta team (loaded with bundle file)
  - With and without conditioning on Beethoven Dataset
5. Music Transformer Model 
  - Based on Google Colab file publish by the Magenta team here: (https://colab.research.google.com/notebooks/magenta/piano_transformer/piano_transformer.ipynb#scrollTo=QI5g-x4foZls)
  - Due to unique TensorFlow requirements the generation of Midi's with the Music Transformer must be done in a different Colab File than Polyphony Rnn and Performance Rnn
  - The samples played here are generated in the Transformer Colab File published by Google Magenta.

In [ ]:
#@title Setup Magenta Environment and Import Necessary dependencies
#@markdown Define soundfont to specify sounds used by Midi files
#@markdown - We use Yamaha C5 Salamander Soundfont

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Copying Salamander piano SoundFont (via https://sites.google.com/site/soundfonts4u) from GCS...')
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/

import note_seq
import os


Copying Salamander piano SoundFont (via https://sites.google.com/site/soundfonts4u) from GCS...


In [ ]:
#@title Import Dependencies From Magenta
#@markdown Import libraries from Magenta, Tensorflow and Numpy

#!pip install numba==0.48

from google.colab import files
import numpy as np
import os
import tensorflow as tf
import magenta.music as mm
import magenta
from magenta.scripts import convert_dir_to_note_sequences
from magenta.models.polyphony_rnn import *
from magenta.models.performance_rnn import *

import datetime, os

#1) Download Required Files from Github 
- Model Bundle Files (.mag) and Midi Sample Filesfrom Github
- Bundles Used:
  - polyphony.mag
  - performance.mag
  - performance_with_dynamics.mag
  - perf_with_dynamics_maestro_base.mag
  - beeth_perf_with_dynamics_base.mag
  - beeth_perf_with_dynamics_pretrained_Magenta.mag
  - beeth_performance_Magenta.mag
- Midi Sample Files Used (any Midi can be uploaded to replace these:
  - Fur_Elise.mid
  - Moonlight_Sonate_No.14.mid
  - Sonate_No30_E_majorOp109.mid
  - Bach_prelude_no1_in_C.mid

The rest of this tutorial assumes these files have been downloaded.

In [ ]:
#@title Run this cell to download the first three bundle files from Google

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


note_seq.notebook_utils.download_bundle('performance.mag', '/content/')
note_seq.notebook_utils.download_bundle('performance_with_dynamics.mag', '/content/')

In [ ]:
! curl -L http://download.magenta.tensorflow.org/models/polyphony_rnn.mag  --output polyphony_rnn.mag

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6497k  100 6497k    0     0  19.1M      0 --:--:-- --:--:-- --:--:-- 19.1M


In [ ]:
#@title The model bundles that we have trained can be downloaded via the Google Drive share links below
#@markdown - This is needed as the files are too large for Github 
#@markdown - https://drive.google.com/file/d/1-N2TXgK5jnTv75CMvTR36eTz25NKHhFq/view?usp=sharing
#@markdown - https://drive.google.com/file/d/1pSwNs9oulYY_74HYJQ_qQoWtuf5dgn6s/view?usp=sharing
#@markdown - https://drive.google.com/file/d/1VXYLS6bfZa1IBS5-eNcs4swpy0-eoKuG/view?usp=sharing
#@markdown - https://drive.google.com/file/d/1wUGM--z8tfH0RGLzTyWoUWXvba1X2rkK/view?usp=sharing 

#@markdown The files need to next be uploaded to the Google Colab environment using the next cell

In [ ]:
#@title Run to Upload the four .mag files from the previous cell
from google.colab import files
uploaded = files.upload()

In [182]:
#@title Download Primer Midi Files from GitHub Here
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/primer_midi/Fur_Elise.mid --output Fur_Elise.mid
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/primer_midi/Moonlight_Sonate_No.14.mid --output Moonlight_Sonate_No.14.mid 
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/primer_midi/Sonate_No30_E_majorOp109.mid --output Sonate_No30_E_majorOp109.mid
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/primer_midi/Bach_prelude_no1_in_C.mid --output Bach_prelude_no1_in_C.mid 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174  100   174    0     0   1641      0 --:--:-- --:--:-- --:--:--  1641
100 14264  100 14264    0     0  92623      0 --:--:-- --:--:-- --:--:-- 92623
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   187  100   187    0     0   1764      0 --:--:-- --:--:-- --:--:--  1764
100 10942  100 10942    0     0  68387      0 --:--:-- --:--:-- --:--:-- 68387
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   189  100   189    0     0   1928      0 --:--:-- --:--:-- --:--:--  1928
100 81514  100 81514    0     0   324k      0 --:--:-- --:--:-- --:--:--  324k
  % Total    % Received % Xferd  Average Speed   Tim

#Define Primer Sequence for Models to Generate Samples From 
 

In [ ]:
#@title More Dependencies Needed for Sequence Generation
#@markdown - Define upload_midi() to convert individual Midi to NoteSequence
#@markdown - SF2_PATH=path to sound font
#@markdown - SAMPLE_RATE = 16000, this can be changed 
SF2_PATH = '/content/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

# Upload a MIDI file and convert to NoteSequence.
def upload_midi():
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return note_seq.midi_to_note_sequence(data[0])

from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.polyphony_rnn import polyphony_sequence_generator

from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [183]:
#@title Select Midi Primer and truncate if needed
#@markdown - Choose desired Midi and length and then run this cell
#@markdown - A shorter primer is often better
#@markdown - Select "Upload your own" to use a new Midi file
filenames = {
    'Beethoven Fur Elise': '/content/Fur_Elise.mid',
    'Beethoven Moonlight Sonate': '/content/Moonlight_Sonate_No.14.mid',
    'Beethoven Sonate No. 30 E Major': '/content/Sonate_No30_E_majorOp109.mid',
    'Bach Prelude No.1 in C': '/content/Bach_prelude_no1_in_C.mid'
}
primer = 'Beethoven Fur Elise'  #@param ['Beethoven Fur Elise', 'Beethoven Moonlight Sonate', 'Beethoven Sonate No. 30 E Major', 'Bach Prelude No.1 in C', 'Upload your own!']

if primer == 'Upload your own!':
  primer_ns = upload_midi()
else:
  # Use one of the provided primers.
  primer_ns = note_seq.midi_file_to_note_sequence(filenames[primer])
  #primer_full = primer_ns.copy()
# Handle sustain pedal in the primer.
primer_ns = note_seq.apply_sustain_control_changes(primer_ns)

# Trim to desired number of seconds.
max_primer_seconds = 10  #@param {type:"slider", min:1, max:120}
if primer_ns.total_time > max_primer_seconds:
  print('Primer is longer than %d seconds, truncating.' % max_primer_seconds)
  primer_ns = note_seq.extract_subsequence(
      primer_ns, 0, max_primer_seconds)

# Remove drums from primer if present.
if any(note.is_drum for note in primer_ns.notes):
  print('Primer contains drums; they will be removed.')
  notes = [note for note in primer_ns.notes if not note.is_drum]
  del primer_ns.notes[:]
  primer_ns.notes.extend(notes)

# Set primer instrument and program.
for note in primer_ns.notes:
  note.instrument = 1
  note.program = 0

# Play and plot the primer.
note_seq.play_sequence(
    primer_ns,
    synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(primer_ns)


Primer is longer than 10 seconds, truncating.


#2) Generate Samples for Polyphony Rnn

In [151]:
from magenta.models.polyphony_rnn.polyphony_sequence_generator import PolyphonyRnnSequenceGenerator

#@title Generate Sequences with Polyphony Pre-Trained Model from Example Midi
#@markdown - Set to extend sequence to 30 seconds including length of primer
#@markdown - May have issues generating samples from primers with changing tempos. Might raise an error if other models have been run in this Colab 
#@markdown - Temperature argument determines the randomness of sample. The default value is 1.0 and represents softmax probabilities that can be changed in this cell
poly_bundle = sequence_generator_bundle.read_bundle_file("/content/polyphony_rnn.mag")
generator_map = polyphony_sequence_generator.get_generator_map()
generator = generator_map['polyphony'](checkpoint=None, bundle=poly_bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 1.0 is default. 
generator_options.args['condition_on_primer'].bool_value= False #Cannot use this unless the primer is a midi with only 1 tempo
generator_options.args['no_inject_primer_during_generation'].bool_value = True #False plays the primer sequence while playing beginning of sample (allows harmonization)
generate_section = generator_options.generate_sections.add(start_time=primer_ns.total_time, end_time=30) #Sample is continuation so start must be equal to end of primer, stop indicates end of sample
sequence = generator.generate(primer_ns, generator_options)

#Play and Plot generated sequence
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sequence)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


INFO:tensorflow:Restoring parameters from /tmp/tmpcy64ktrs/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmpcy64ktrs/model.ckpt


MultipleTempoError: ignored

In [152]:
#@title (Optional) Name and Download Generated Sample Midi from Sequence Generator
#@markdown - Will save most recent generated sample from any model
#@markdown - Default name for this file is "polyphony.mid but can be changed within this cell"
note_seq.sequence_proto_to_midi_file(sequence, '/content/polyphony.mid')
files.download('/content/polyphony.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 3) Generate Samples for Performance Rnn 
- "performance" configuration

In [153]:
#@title Generate Sequences with Performance Rnn Pre-Trained Model from Example Midi
#@markdown - This is the default pre-trained model released by Magenta and trained on the MAESTRO dataset 
#@markdown - Bundle File: "performance.mag"
performance_bundle = sequence_generator_bundle.read_bundle_file("/tmp/performance.mag")
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map['performance'](checkpoint=None, bundle=performance_bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 0.90 is default. 
#generator_options.args['condition_on_primer'].bool_value= False #Cannot use this unless the primer is a midi with only 1 tempo
#generator_options.args['no_inject_primer_during_generation'].bool_value = True #False plays the primer sequence while playing beginning of sample (allows harmonization)
generate_section = generator_options.generate_sections.add(start_time=primer_ns.total_time, end_time=30) #Sample is continuation so start must be equal to end of primer, stop indicates end of sample
sequence = generator.generate(primer_ns, generator_options)

#Play and Plot generated sequence
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sequence)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


INFO:tensorflow:Restoring parameters from /tmp/tmp1jfz_pdi/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmp1jfz_pdi/model.ckpt


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -2884.571045 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -2884.571045 


In [141]:
#@title (Optional) Name and Download Generated Sample Midi
#@markdown - Will save most recent generated sample from any model
#@markdown - Default name for this file is "performance_Magenta.mid but can be changed within this cell"
note_seq.sequence_proto_to_midi_file(sequence, '/content/performance_Magenta.mid')
files.download('/content/performance_Magenta.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 4) Generate Samples for Performance Rnn with Dynamics
- "performance_with_dynamics" & "performance_with_dynamics_compact" configurations
- Both configurations use the same information but perform one-hot encoding at different times.

In [154]:
#@title Generate Sequences with Performance Rnn Pre-Trained Model from Example Midi
#@markdown - Trained by us on the entire MAESTRO dataset
#@markdown - Configuration = "performance_with_dynamics_compact"
#@markdown - Bundle File = "perf_with_dynamics_maestro_base.mag"
#@markdown - Model lacks long-term coherence and tends to leave hanging notes that can go as long as 5 seconds.

performance_beeth_bundle = sequence_generator_bundle.read_bundle_file("/content/perf_with_dynamics_maestro_base.mag")
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map['performance_with_dynamics_compact'](checkpoint=None, bundle=performance_beeth_bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 0.90 is default. 
generator_options.args['condition_on_primer'].bool_value= True #Cannot use this unless the primer is a midi with only 1 tempo
generator_options.args['no_inject_primer_during_generation'].bool_value = True #False plays the primer sequence while playing beginning of sample (allows harmonization)
generate_section = generator_options.generate_sections.add(start_time=primer_ns.total_time, end_time=30) #Sample is continuation so start must be equal to end of primer, stop indicates end of sample
sequence = generator.generate(primer_ns, generator_options)

#Play and Plot generated sequence
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sequence)

INFO:tensorflow:Restoring parameters from /tmp/tmp0obhvb6r/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmp0obhvb6r/model.ckpt


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -3058.565430 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -3058.565430 


In [143]:
#@title (Optional) Name and Download Generated Sample Midi
#@markdown - Will save most recent generated sample from any model
#@markdown - Default name for this file is "perf_with_dynamics_maestro_base.mid but can be changed within this cell"
note_seq.sequence_proto_to_midi_file(sequence, '/content/perf_with_dynamics_maestro_base.mid')
files.download('/content/perf_with_dynamics_maestro_base.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [158]:
#@title Generate Sequences with Performance Rnn Pre-Trained Model Conditioned on from Example Midi
#@markdown - Trained by us on the entire MAESTRO dataset and conditionally trained again on a second dataset of Beethoven Music
#@markdown - Bundle File = "beeth_perf_with_dynamics_base.mag"
#@markdown - Configuration = "performance_with_dynamics_compact"
#@markdown - Model lacks long-term coherence and tends to leave hanging notes that can go as long as 5 seconds.

performance_beeth_bundle = sequence_generator_bundle.read_bundle_file("/content/beeth_perf_with_dynamics_base.mag")
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map['performance_with_dynamics_compact'](checkpoint=None, bundle=performance_beeth_bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 0.90 is default. 
generator_options.args['condition_on_primer'].bool_value= True #Cannot use this unless the primer is a midi with only 1 tempo
generator_options.args['no_inject_primer_during_generation'].bool_value = True #False plays the primer sequence while playing beginning of sample (allows harmonization)
generate_section = generator_options.generate_sections.add(start_time=primer_ns.total_time, end_time=30) #Sample is continuation so start must be equal to end of primer, stop indicates end of sample
sequence = generator.generate(primer_ns, generator_options)

#Play and Plot generated sequence
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sequence)

INFO:tensorflow:Restoring parameters from /tmp/tmp8c6oeurf/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmp8c6oeurf/model.ckpt


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1971.697266 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1971.697266 


In [145]:
#@title (Optional) Name and Download Generated Sample Midi
#@markdown - Will save most recent generated sample from any model
#@markdown - Default name for this file is "beeth_perf_with_dynamics_base.mid but can be changed within this cell"
note_seq.sequence_proto_to_midi_file(sequence, '/content/beeth_perf_with_dynamics_base.mid')
files.download('/content/beeth_perf_with_dynamics_base.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [162]:
#@title Generate Sequences with Performance Rnn Pre-Trained Model from Example Midi
#@markdown - Pre-Trained model bundle from Magenta Github trained on the entire MAESTRO dataset
#@markdown - Bundle File = "performance_with_dynamics.mag"
#@markdown - Configuration = "performance_with_dynamics"
#@markdown - Model lacks long-term coherence and tends to leave hanging notes that can go as long as 5 seconds.

performance_beeth_bundle = sequence_generator_bundle.read_bundle_file("/content/performance_with_dynamics.mag")
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map['performance_with_dynamics'](checkpoint=None, bundle=performance_beeth_bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 0.90 is default. 
generator_options.args['condition_on_primer'].bool_value= True #Cannot use this unless the primer is a midi with only 1 tempo
generator_options.args['no_inject_primer_during_generation'].bool_value = True #False plays the primer sequence while playing beginning of sample (allows harmonization)
generate_section = generator_options.generate_sections.add(start_time=primer_ns.total_time, end_time=30) #Sample is continuation so start must be equal to end of primer, stop indicates end of sample
sequence = generator.generate(primer_ns, generator_options)

#Play and Plot generated sequence
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sequence)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


INFO:tensorflow:Restoring parameters from /tmp/tmpk1xx096i/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmpk1xx096i/model.ckpt


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -3189.083496 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -3189.083496 


In [147]:
#@title (Optional) Name and Download Generated Sample Midi
#@markdown - Will save most recent generated sample from any model
#@markdown - Default name for this file is "performance_with_dynamics.mid but can be changed within this cell"
note_seq.sequence_proto_to_midi_file(sequence, '/content/performance_with_dynamics.mid')
files.download('/content/performance_with_dynamics.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [163]:
#@title Generate Sequences with Open-Source Performance Rnn Pre-Trained Model from Example Midi Conditioned on Beethoven Dataset
#@markdown - Pre-Trained model bundle from Magenta Github initially trained on the entire MAESTRO dataset
#@markdown - Conditionally trained by us on a second dataset of Beethoven Music using the "performance_with_dynamics.mag" bundle as a warm-up file
#@markdown - Bundle File = "beeth_performance_Magenta.mag"
#@markdown - Initial Training Configuration = "performance_with_dynamics"
#@markdown - Conditional Training Configuration = "performance_with_dynamics_compact" is used to reduce training file size of Beethoven Midis
#@markdown - Model lacks long-term coherence and tends to leave hanging notes that can go as long as 5 seconds.

performance_beeth_bundle = sequence_generator_bundle.read_bundle_file("/content/beeth_performance_Magenta.mag")
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map['performance_with_dynamics_compact'](checkpoint=None, bundle=performance_beeth_bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 0.90 is default. 
generator_options.args['condition_on_primer'].bool_value= False #Cannot use this unless the primer is a midi with only 1 tempo
generator_options.args['no_inject_primer_during_generation'].bool_value = True #False plays the primer sequence while playing beginning of sample (allows harmonization)
generate_section = generator_options.generate_sections.add(start_time=primer_ns.total_time, end_time=30) #Sample is continuation so start must be equal to end of primer, stop indicates end of sample
sequence = generator.generate(primer_ns, generator_options)

#Play and Plot generated sequence
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sequence)

INFO:tensorflow:Restoring parameters from /tmp/tmpgg9ep0oy/model.ckpt


INFO:tensorflow:Restoring parameters from /tmp/tmpgg9ep0oy/model.ckpt


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Need to generate 1999 more steps for this sequence, will try asking for 1200 RNN steps


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1180.771973 


INFO:tensorflow:Beam search yields sequence with log-likelihood: -1180.771973 


In [149]:
#@title (Optional) Name and Download Generated Sample Midi
#@markdown - Will save most recent generated sample from any model
#@markdown - Default name for this file is "beeth_perf_with_dynamics_Magenta.mid but can be changed within this cell"
note_seq.sequence_proto_to_midi_file(sequence, '/content/beeth_perf_with_dynamics_Magenta.mid')
files.download('/content/beeth_perf_with_dynamics_Magenta.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 5) Generate samples with Music Transformer
- Midi files are generated using the Music Transformer for each primer file 
- Generating Piano Music with the Transformer Google Colab file at https://colab.research.google.com/notebooks/magenta/piano_transformer/piano_transformer.ipynb#scrollTo=3URxzTQyXfdO 
- Separate Colab file must be used as the Transformer model requires Tensorflow 1 while the rest of the models exhibited require Tensorflow 2.


In [173]:
#@title Download generated Midi Samples from GitHub
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/Generated_Sample_Midis/Fur_Elise/transformer_fur_elise.mid --output transformer_fur_elise.mid
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/Generated_Sample_Midis/Moonlight_Sonate/transformer_moonlight_sonate.mid --output transformer_moonlight_sonate.mid
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/Generated_Sample_Midis/Sonate_No.30_E_major/transformer_Sonate_No.30_E_major.mid --output transformer_sonate_no30.mid
!curl -L https://github.com/CoryShounick/DSCI-552---Final-Project-Files/raw/main/Generated_Sample_Midis/Bach_prelude_no1_in_C/transformer_Bach_prelude_no1.mid --output transformer_bach_prelude_no1.mid

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   207  100   207    0     0   1990      0 --:--:-- --:--:-- --:--:--  1990
100  7220  100  7220    0     0  47500      0 --:--:-- --:--:-- --:--:-- 47500
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   221  100   221    0     0   2125      0 --:--:-- --:--:-- --:--:--  2125
100  1715  100  1715    0     0   7175      0 --:--:-- --:--:-- --:--:--     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   229  100   229    0     0   2201      0 --:--:-- --:--:-- --:--:--  2201
100  1582  100  1582    0     0   5221      0 --:--:-- --:--:-- --:--:--  5221
  % Total    % Received % Xferd  Average Speed   Tim

In [177]:
#@title Play and Plot Fur Elise Generated Sample
fur_elise=note_seq.midi_file_to_note_sequence('/content/transformer_fur_elise.mid')
note_seq.play_sequence(fur_elise, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(fur_elise)

In [178]:
#@title Play and Plot Moonlight Sonate Generated Sample
moonlight=note_seq.midi_file_to_note_sequence('/content/transformer_moonlight_sonate.mid')
note_seq.play_sequence(moonlight, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(moonlight)

In [179]:
#@title Play and Plot Sonate No. 30 in E Major Generated Sample
sonate_no30=note_seq.midi_file_to_note_sequence('/content/transformer_sonate_no30.mid')
note_seq.play_sequence(sonate_no30, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(sonate_no30)

In [180]:
#@title Play and Plot Bach Prelude No1 in C Generated Sample
bach_prelude=note_seq.midi_file_to_note_sequence('/content/transformer_bach_prelude_no1.mid')
note_seq.play_sequence(bach_prelude, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(bach_prelude)